In [ ]:
import imageio
import glob
import os
import time
import cv2
import tensorflow as tf
from tensorflow.keras import layers
from IPython import display
import matplotlib.pyplot as plt
import numpy as np
%matplotlib inline
from tensorflow import keras
import pydicom as dicom
from skimage.util import view_as_windows
from skimage import exposure, io, filters
import scipy
from scipy.linalg import svd
import matplotlib.patches as patches
from PIL import Image
from sklearn.model_selection import train_test_split
from scipy.spatial import distance
from sklearn.cluster import DBSCAN
from sklearn import metrics
from sklearn.decomposition import PCA
from sklearn.cluster import KMeans
from sklearn.cluster import AgglomerativeClustering
from skimage.feature import local_binary_pattern
import pandas as pd
from typing import AnyStr, BinaryIO, Dict, List, NamedTuple, Optional, Union
from skimage.exposure import rescale_intensity
import pydicom
import pydicom.encaps

In [ ]:
def _get_image_laterality(pixel_array: np.ndarray) -> str:
    left_edge = np.sum(pixel_array[:, 0])  # sum of left edge pixels
    right_edge = np.sum(pixel_array[:, -1])  # sum of right edge pixels
    return "R" if left_edge < right_edge else "L"


def _get_window_center(ds: dicom.dataset.FileDataset) -> np.float32:
    return np.float32(ds[0x5200, 0x9229][0][0x0028, 0x9132][0][0x0028, 0x1050].value)


def _get_window_width(ds: dicom.dataset.FileDataset) -> np.float32:
    return np.float32(ds[0x5200, 0x9229][0][0x0028, 0x9132][0][0x0028, 0x1051].value)


def read_cvs_file(path):
    csv_matrix = np.genfromtxt(path, delimiter=',', skip_header=1)
    return csv_matrix

def dcmread_image(
    fp: Union[str, "os.PathLike[AnyStr]", BinaryIO],
    view: str, paziente: str, 
    index: Optional[np.uint] = None,
) -> np.ndarray:
    """Read pixel array from DBT DICOM file"""
    ds = dicom.dcmread(fp)
    print("considero: ", ds.PatientID, paziente)
    ds.decompress(handler_name="pylibjpeg")
    pixel_array = ds.pixel_array
    view_laterality = view[0].upper()
    print("view_laterality: ", view_laterality)
    image_laterality = _get_image_laterality(pixel_array[index or 0])
    print("image_laterality: ", image_laterality)

    if index is not None:
        pixel_array = pixel_array[index]
    if image_laterality[0] != view_laterality[0]:
        pixel_array = np.flip(pixel_array, axis=(-1, -2))
        pixel_array = (pixel_array / np.max(pixel_array)) * 255.
        print("flip")# Normalizza i valori dei pixel
    else:
        pixel_array = (pixel_array / np.max(pixel_array)) * 255.
        print("no flip")
    return pixel_array

In [ ]:
training_daAumentare = "your path"
path_masse_dir = "your path/"
path_label = "your path to BCS-DBT boxes-train-v2.csv"
ext = ".png"
extension = ".dcm"
boxes_train_csv = pd.read_csv(path_label)
row = 1

In [ ]:
def getImagesFilenames(rootdir):
    filelist = []
    for subdir, dirs, files in os.walk(rootdir):
        for file in files:
            if file.endswith(extension):
                filepath = os.path.join(subdir, file)
                filelist.append(filepath)

    return filelist

In [ ]:
def empty_directory(path):
    if os.path.exists(path):
        # Cancella la directory esistente e tutti i suoi contenuti
        for root, dirs, files in os.walk(path, topdown=False):
            for file in files:
                os.remove(os.path.join(root, file))
            for dir in dirs:
                os.rmdir(os.path.join(root, dir))

In [ ]:
def truncation_normalization(img, mask):
    """
    Clip and normalize pixels in the breast ROI.
    @img : numpy array image
    @mask : numpy array mask of the breast
    return: numpy array of the normalized image
    """
    Pmin = np.percentile(img[mask!=0], 0.1)
    Pmax = np.percentile(img[mask!=0], 99)
    truncated = np.clip(img,Pmin, Pmax)  
    normalized = (truncated - Pmin)/(Pmax - Pmin)
    normalized[mask==0]=0
    return normalized

In [ ]:
import time

def select_mass(filename):
    output_folder = "your path"  # Aggiorna con il percorso della cartella in cui salvare le slice_prep
    empty_directory(output_folder)
    number_of_Frames=[]
    index = 0
    
    for file in filename:
        slices_prep = []
        immagini = []
        try:
            print("file: ", file)
            ds = dicom.dcmread(file)
            #rimuovo l'indice nell'ipotesi che se ho due immagini nella stessa view, mi serve un solo flip
            patient_row = boxes_train_csv[(boxes_train_csv['PatientID'] == ds.PatientID) & (boxes_train_csv['VolumeSlices'] == ds.NumberOfFrames)]
            view = patient_row['View'].values[0]
            patient = patient_row['PatientID'].values[0]
            data = dcmread_image(file, view, patient)

            # data = np.uint8(data)
            numSlice = data.shape[0]
            slice_to_take = patient_row['Slice'].values
            print('slice_to_take', slice_to_take)
            original_coordinate = patient_row[['X', 'Y', 'Width', 'Height']].values[0]
            x_r = original_coordinate[0]
            y_r = original_coordinate[1]
            w_r = original_coordinate[2]
            h_r = original_coordinate[3]
            target_width = 200
            target_height = 100

            for slices in slice_to_take:
                for j in range(numSlice):    
                    if j == slices-1:
                        print("slice selected: ", j)
                        data_jm1 = np.uint16(data[j-1])
                        data_np_j = np.uint16(data[j])
                        data_jp1 = np.uint16(data[j+1])
                        data_jm1 = data_jm1[y_r:y_r+h_r, x_r:x_r+w_r]
                        data_np_j = data_np_j[y_r:y_r+h_r, x_r:x_r+w_r]
                        data_jp1 = data_jp1[y_r:y_r+h_r, x_r:x_r+w_r]
                        print(data_np_j.shape)

                        immagini.append(data_jm1)
                        immagini.append(data_np_j)
                        immagini.append(data_jp1)
                        for img in immagini:
                            if w_r > 200 or h_r > 100:
                                _, binary_image = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                                print(binary_image.shape)
                                norm = truncation_normalization(img, binary_image)
                                norm = tf.expand_dims(norm, axis=-1)
                                norm = tf.image.resize(norm, [100, 200], method=tf.image.ResizeMethod.BICUBIC)
                                norm = np.array(norm, dtype=np.float64)

                                print("norm shape: ", norm.shape)

                            else:
                                # Calcola il padding orizzontale (se necessario)
                                if w_r < target_width:
                                    diff_width = target_width - w_r
                                    left_padding = diff_width // 2
                                    right_padding = diff_width - left_padding
                                else:
                                    left_padding = 0
                                    right_padding = 0

                                # Calcola il padding verticale (se necessario)
                                if h_r < target_height:
                                    diff_height = target_height - h_r
                                    top_padding = diff_height // 2
                                    bottom_padding = diff_height - top_padding
                                else:
                                    top_padding = 0
                                    bottom_padding = 0

                                # Applica il padding utilizzando la funzione cv2.copyMakeBorder
                                norm = cv2.copyMakeBorder(img, top_padding, bottom_padding, left_padding, right_padding, cv2.BORDER_CONSTANT, value=(0, 0, 0))
                                norm = np.uint16(norm)
                                _, binary_image = cv2.threshold(norm, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                                print(binary_image.shape)
                                norm = truncation_normalization(norm, binary_image)
                                norm = tf.expand_dims(norm, axis=-1)

                            norm = tf.convert_to_tensor(norm, dtype=tf.float64)

                            slices_prep.append(norm/255.)
                            number_of_Frames.append((ds.PatientID, ds.NumberOfFrames,ds.ViewPosition))
            
            
            
            # Crea una cartella distinta utilizzando il patientID come nome
            patient_id = ds.PatientID
            fol_name = f"{patient_id}"
            output_folder_ = os.path.join(output_folder, fol_name)
            os.makedirs(output_folder_, exist_ok=True)

            for idx, slice_pr in enumerate(slices_prep):

                timestamp = str(time.time())

                file_name = f"slice_{index}_{idx}_{timestamp}.png"
                index = index +1
                # Convert the processed slice tensor to a NumPy array
                image_data = (slice_pr.numpy()*65535).astype(np.float64)
                plt.imshow(image_data, cmap='gray')

                # Save the DICOM image
                output_file_path = output_folder_ + "/" + file_name
                cv2.imwrite(output_file_path, image_data)

                print(f"Saved processed slice {idx} as DICOM image: {output_file_path}")
        except Exception as e:
            print(f"Errore: {str(e)}")
    return number_of_Frames        

In [ ]:
mass_filenames = getImagesFilenames(path_masse_dir)
dicom_informations = select_mass(mass_filenames)


In [ ]:
print(len(dicom_informations))


In [ ]:
import cv2
import random
import math
# Funzione per applicare l'augmentation all'immagine
def apply_data_augmentation(image):
    augmented_images = []
    augmentation_types = []  # Aggiungi questa lista per tenere traccia del tipo di augmentation
    angles = []
    '''
    # Esegui il flip orizzontale
    flipped_image = cv2.flip(image, 0)
    augmented_images.append(flipped_image)
    augmentation_types.append('flip_horizontal')  # Aggiungi il tipo di augmentation
    '''
    # Esegui la rotazione 
   
    angle = 30  # Angolo di rotazione casuale tra -30 e 30 gradi
    M = cv2.getRotationMatrix2D((image.shape[1] // 2, image.shape[0] // 2), angle, 1)
    rotated_image = cv2.warpAffine(image, M, (image.shape[1], image.shape[0]))
    augmented_images.append(rotated_image)
    augmentation_types.append('rotate_random')
    print("ruoto")
    angles.append(angle)
    
    print("aumento")
    return augmented_images, augmentation_types, angles




In [ ]:
def augment_and_update_csv(rootdir, nof):
    augmented_data = []
    index = 200
    for subdir, dirs, files in os.walk(rootdir):
        files = sorted(files, key=extract_numbers)
        # PER OGNI IMMAGINE
        for file in files:
            if file.endswith(ext):
                filepath = os.path.join(subdir, file)
                img = Image.open(filepath)
                patientID = nof[0][0]
                nOFrames = nof[0][1]
                viewPosition = nof[0][2]
                print("considero: ", patientID)
                # Converte l'immagine in un array NumPy
                data = np.array(img)
                patient_row = boxes_train_csv[(boxes_train_csv['PatientID'] == patientID)]
                print("noframes: ", nOFrames)
                print(patient_row)
                nof.pop(0)

                if not patient_row.empty:
                    label = patient_row['Class'].values[0]
                    print("label: ", label)                        
                    # Applica la data augmentation all'immagine
                    augmented_images, augmentation_type, angles  = apply_data_augmentation(data)
                    augmented_data.append([patientID,viewPosition, label])

                    # Aggiungi i dati originali alla lista
                    for i in range(len(augmented_images)):
                        augmented_data.append([patientID,viewPosition, label])

                    
                    for idx, augmented_image in enumerate(augmented_images):
                        file_name = f"{index}.png"
                        index = index+1
                        output_file_path = os.path.join(subdir, file_name)  # Percorso completo del file di output
                        augmented_image = Image.fromarray(augmented_image)  # Converti l'array NumPy in un oggetto immagine
                        augmented_image.save(output_file_path)
                        print(f"Saved processed slice {idx} as DICOM image: {output_file_path}")
                           
                    
    #augmented_data = [list(t) for t in augmented_data]
    
    # Crea un nuovo DataFrame con i dati aumentati
    augmented_df = pd.DataFrame(augmented_data)

    # Salva il nuovo DataFrame in un nuovo CSV
    augmented_df.to_csv('your path to ROI_AUGMENTED.csv', index=False)
                

In [ ]:
numberOfFr = pd.read_csv('your path to info_dicom_roi.csv')
list_of_tuples = [tuple(x) for x in numberOfFr.to_records(index=False)]

In [ ]:
for item in list_of_tuples:
    print(item)

In [ ]:
dir_masse_da_aumentare = "E:/ROI_Slices/"


In [ ]:
import re
augment_and_update_csv(training_daAumentare, list_of_tuples)

In [ ]:
data_and_labels = pd.read_csv('your path to ROI_AUGMENTED.csv')
aug_tuples = [tuple(x) for x in data_and_labels.to_records(index=False)]
dir_masse_aumentate = "your path"

In [ ]:
for item in aug_tuples:
    print(item)

In [ ]:
def check_for_separability(rootdir, nof):
    dataset = []
    index_csv = 0
    for subdir, dirs, files in os.walk(rootdir):
            # PER OGNI IMMAGINE
            for file in files:
                if file.endswith(ext):
                    filepath = os.path.join(subdir, file)
                    img = Image.open(filepath)
                    patientID = nof[0][0]
                    nOFrames = nof[0][1]
                    viewPosition = nof[0][2]
                    print("considero: ", patientID)
                    # Converte l'immagine in un array NumPy
                    data = np.array(img)
                    patient_row = data_and_labels[(data_and_labels['PatientID'] == patientID) &(data_and_labels['INDEX'] == index_csv)]
                    print("indice csv: ", index_csv)
                    print(patient_row)
                    index_csv = index_csv + 1
                    nof.pop(0)

                    if not patient_row.empty:
                        label = patient_row['Class'].values[0]
                        dataset.append((patientID, label, data))
    return dataset

In [ ]:
dataset_complete = check_for_separability(dir_masse_aumentate, aug_tuples)

In [ ]:
print(type(dataset_complete[1]))

In [ ]:
train_images = [item[2] for item in dataset_complete]
train_labels = [item[1] for item in dataset_complete]

In [ ]:
#SEPARABILITA' TRA MASSE BENIGNE E MALIGNE
import numpy as np
import matplotlib.pyplot as plt

coordinates = []

for img in train_images:
    img_array = np.array(img)  # Converti l'immagine in un array NumPy
    coords = img_array.flatten()  # Appiattisci l'immagine in un singolo array di coordinate
    coordinates.extend(coords.tolist())  # Aggiungi le coordinate alla lista

# Separazione delle coordinate per classe
class_0_coords = [coordinates[i] for i in range(len(train_labels)) if train_labels[i] == 0]
class_1_coords = [coordinates[i] for i in range(len(train_labels)) if train_labels[i] == 1]

# Ora puoi creare il grafico di dispersione
plt.scatter(class_0_coords, class_0_coords, label='Classe 0', c='blue')
plt.scatter(class_1_coords, class_1_coords, label='Classe 1', c='red')

plt.xlabel('Coordinate X e Y')
plt.ylabel('Coordinate X e Y')

plt.legend()
plt.show()


In [ ]:
import random
def extract_ROI_nomass(filename):
    output_folder = "your path"  # Aggiorna con il percorso della cartella in cui salvare le slice_prep
    empty_directory(output_folder)
    number_of_Frames=[]
    index = 100
    for file in filename:
        if file.endswith(extension):
            slices_prep = []
            immagini = []
            try:
                print("file: ", file)
                ds = dicom.dcmread(file)
                #rimuovo l'indice nell'ipotesi che se ho due immagini nella stessa view, mi serve un solo flip
                ds = dicom.dcmread(file)        
                print(f'considero: {ds.PatientID}')
                ds.decompress(handler_name="pylibjpeg")
                numSlice = ds.NumberOfFrames
                data = ds.pixel_array
                data = data.astype(np.float32)
                data_jm1 = data[19]
                data_np_j = data[20]
                data_jp1 = data[21]
                data_jm1 = np.uint16(data_jm1)
                data_np_j = np.uint16(data_np_j)
                data_jp1 = np.uint16(data_jp1)
                
                immagini.append(data_jm1)
                immagini.append(data_np_j)
                immagini.append(data_jp1)
                x_r = random.randint(0, 100)
                y_r = random.randint(0,200)
                for img in immagini:
                    _, binary_image = cv2.threshold(img, 0, 255, cv2.THRESH_BINARY + cv2.THRESH_OTSU)
                    contours, _ = cv2.findContours(np.uint8(binary_image), cv2.RETR_EXTERNAL, cv2.CHAIN_APPROX_SIMPLE)
                    breast_contour = max(contours, key=cv2.contourArea)
                    x, y, w, h = cv2.boundingRect(breast_contour)
                    w = w-100
                    h = h-900
                    cropped_image = img[y:y + h, x:x + w]
                    binary_image = binary_image[y:y + h, x:x + w]
                    cropped_image = truncation_normalization(cropped_image, binary_image)
                    width, height = cropped_image.shape

                    

                    rect = cropped_image[y_r:y_r+100, x_r:x_r+200]



                    norm = tf.expand_dims(rect, axis=-1)

                    norm = tf.convert_to_tensor(norm, dtype=tf.float64)
                    norm = tf.image.resize(norm, [100,200])

                    slices_prep.append(norm/255.)



                # Crea una cartella distinta utilizzando il patientID come nome
                patient_id = ds.PatientID
                fol_name = f"{patient_id}"
                output_folder_ = os.path.join(output_folder, fol_name)
                os.makedirs(output_folder_, exist_ok=True)

                for idx, slice_pr in enumerate(slices_prep):

                    timestamp = str(time.time())

                    file_name = f"{index}.png"
                    index = index +1
                    # Convert the processed slice tensor to a NumPy array
                    image_data = (slice_pr.numpy()*65535).astype(np.float64)
                    # Save the DICOM image
                    output_file_path = output_folder_ + "/" + file_name
                    cv2.imwrite(output_file_path, image_data)

                    print(f"Saved processed slice {idx} as PNG image: {output_file_path}")
            except Exception as e:
                print(f"Errore: {str(e)}")


In [ ]:
path_nomass = "your path"
if os.path.exists(path_nomass):
    print(f"Il percorso {path_nomass} esiste.")
else:
    print(f"Il percorso {path_nomass} non esiste.")


In [ ]:
mass_filenames = getImagesFilenames(path_nomass)
extract_ROI_nomass(mass_filenames)

In [ ]:
import cv2
import random
import math
# Funzione per applicare l'augmentation all'immagine
def apply_data_augmentation(image):
    augmented_images = []
    augmentation_types = []  # Aggiungi questa lista per tenere traccia del tipo di augmentation
    kernel_size = (5, 5)  # Dimensione del kernel del filtro gaussiano
    sigma = random.uniform(3, 6.0)  # Valore casuale per la deviazione standard
    blurred_image = cv2.GaussianBlur(image, kernel_size, sigma)
    augmented_images.append(blurred_image)
    augmentation_types.append('gaussian_blur')
    
    print("aumento")
    return augmented_images, augmentation_types




In [ ]:
#GAUSSIANIZZO TUTTE LE IMMAGINI
import os
import numpy as np
from PIL import Image
import pandas as pd
import re

def augment_nomass(rootdir):
    for subdir, dirs, files in os.walk(rootdir):
        files = sorted(files, key=extract_numbers)
        # PER OGNI IMMAGINE
        for file in files:
            if file.endswith(ext):
                filepath = os.path.join(subdir, file)
                paziente = os.path.basename(subdir)
                img = Image.open(filepath)
                
                data = np.array(img)
                               
                # Applica la data augmentation all'immagine
                augmented_images, augmentation_type  = apply_data_augmentation(data)
                
                for idx, augmented_image in enumerate(augmented_images):
                    augmented_image = Image.fromarray(augmented_image)  # Converti l'array NumPy in un oggetto immagine
                    augmented_image.save(filepath)  # Sovrascrivi l'immagine originale
                    print(f"Saved processed slice {idx} as PNG image: {filepath}")



In [ ]:
def extract_numbers(s):
    numbers = re.findall(r'\d+',s)
    return[int(num)for num in numbers]

In [ ]:
directory = "your path"
augment_nomass(directory)